In [1]:
import trl

In [2]:
trl.__version__

'0.15.1'

## RL4LLM roadmap

- 从 trl 开始学起，框架较为基础和简单；
    - 深入地学习 GRPO，基于 1.5B 复现 R1，复现 aha moments；
        - 大致也基本能搞清楚 RLHF 阶段的 PPO 算法原理，二者在公式上主要只有 adv（advantage）的估计方法不同；
- 后续可以陆陆续续迁移到更现代更多工程性能优化的 RL4LLM 的框架上
    - 比如 veRL 和 OpenRLHF
    - 假如都是零基础，优先 veRL 吧，除非继承而来的项目是 OpenRLHF；
    - OpenRLHF：2405.11143，5k stars；
    - veRL：2409.19256，3.8k stars；

## loss = 0

> loss 为 0 为什么还可以反向传播，更新梯度；

- loss 为 0，不意味着 gradient 为 0
    - $f(w)=(w-1)^2-1$，在 $w=0$ 时，$f(w)=0$，但其实其 gradient 为 -2
        - 梯度 * 学习率 才是 learning 的本质；
    - $w-\eta\cdot g=0-(0.1*-2)=0.2$
- loss 不再是一个好的 monitor 指标，而是 reward

### loss 为 0 不代表 gradient 为 0

In [1]:
import torch

# 情况1: x - x (梯度为0)
x = torch.tensor([3.0], requires_grad=True)
y1 = x - x  # 数学上等价于 0，但计算图保留关联
y1.backward()  # 反向传播计算梯度
print("Gradient for x - x:", x.grad.item())  # 输出 0.0

Gradient for x - x: 0.0


In [2]:
# 清除梯度，准备下一个示例
x.grad.zero_()

# 情况2: x - x.detach() (梯度为1)
y2 = x - x.detach()  # 分离第二个x，使其视为常数
y2.backward()  # 反向传播计算梯度
print("Gradient for x - x.detach():", x.grad.item())  # 输出 1.0

Gradient for x - x.detach(): 1.0


### loss = $\beta kl$

https://github.com/huggingface/open-r1/issues/239#issuecomment-2646297851

- trl grpo
    - $\beta = 0.04$（default）

$$
\mathcal{J}_{GRPO}(\theta) = \mathbb{E}_{q \sim P(Q), \{o_i\}_{i=1}^G \sim \pi_{\theta_{old}}(O|q)} \left[ \frac{1}{G} \sum_{i=1}^G \frac{1}{|o_i|} \sum_{t=1}^{|o_i|} \min \left( \frac{\pi_\theta(o_{i,t}|q, o_{i,<t})}{\pi_{\theta_{old}}(o_{i,t}|q, o_{i,<t})} \hat{A}_{i,t}, \text{clip} \left( \frac{\pi_\theta(o_{i,t}|q, o_{i,<t})}{\pi_{\theta_{old}}(o_{i,t}|q, o_{i,<t})}, 1-\varepsilon, 1+\varepsilon \right) \hat{A}_{i,t} \right) - \beta D_{KL} (\pi_\theta || \pi_{ref}) \right]
$$

- If you are using the GRPO trainer then the old policy is in effect updated every step, this means you just use a detached version of the current policy.
    - 公式中的 $\pi_{\theta_{old}}$ 是 $\pi_\theta$ 的 detach 版（不参与计算图，视为常数）；
    - $r=\frac{\pi_\theta}{\pi_{\theta_{old}}}=1$,
    - $\text{clip}(1, 1-\epsilon, 1+\epsilon)=1$
- $\hat A_{i,t}=\tilde r_i=\frac{r_i-\mu}{\sigma}$ (z score) （token 级别的 adv = output 级别的 reward 组内 z-score 而来）
 

$$
\begin{split}
\mathcal{J}_{GRPO}(\theta)&= \frac{1}{G} \sum_{i=1}^G \frac{1}{|o_i|} \sum_{t=1}^{|o_i|} \min \left( \frac{\pi_\theta(o_{i,t}|q, o_{i,<t})}{\pi_{\theta_{old}}(o_{i,t}|q, o_{i,<t})} \hat{A}_{i,t}, \text{clip} \left( \frac{\pi_\theta(o_{i,t}|q, o_{i,<t})}{\pi_{\theta_{old}}(o_{i,t}|q, o_{i,<t})}, 1-\varepsilon, 1+\varepsilon \right) \hat{A}_{i,t} \right) - \beta D_{KL} (\pi_\theta || \pi_{ref}) \\
&=\frac1G\sum_i^G\frac1{|o_i|}\sum_{t=1}^{|o_i|}\hat A_{i,t} -\frac1G\sum_{i=1}^G\frac1{|o_i|}\sum_{t=1}^{|o_i|}\beta D_{kl}[\pi_\theta|\pi_{ref}]\\
&=\frac1G\sum_i^G\frac1{|o_i|}\sum_{t=1}^{|o_i|}\hat A_i -\frac1G\sum_{i=1}^G\frac1{|o_i|}\sum_{t=1}^{|o_i|}\beta D_{kl}[\pi_\theta|\pi_{ref}]\\
&=\frac1G\sum_i^G\frac1{|o_i|} {|o_i|}\cdot \hat A_i -\frac1G\sum_{i=1}^G\frac1{|o_i|}\sum_{t=1}^{|o_i|}\beta D_{kl}[\pi_\theta|\pi_{ref}]\\
&=\frac1G\sum_i^G\hat A_i-\frac1G\sum_{i=1}^G\frac1{|o_i|}\sum_{t=1}^{|o_i|}\beta D_{kl}[\pi_\theta|\pi_{ref}]\\
&=\frac1G\sum_i^G\frac{r_i-\mu}{\sigma}-\frac1G\sum_{i=1}^G\frac1{|o_i|}\sum_{t=1}^{|o_i|}\beta D_{kl}[\pi_\theta|\pi_{ref}]\\
&=\frac{\sum_i r_i-G\mu}{G}-\frac1G\sum_{i=1}^G\frac1{|o_i|}\sum_{t=1}^{|o_i|}\beta D_{kl}[\pi_\theta|\pi_{ref}]\\
&= 0 -\frac1G\sum_{i=1}^G\frac1{|o_i|}\sum_{t=1}^{|o_i|}\beta D_{kl}[\pi_\theta|\pi_{ref}]\\
&=-\frac1G\sum_{i=1}^G\frac1{|o_i|}\sum_{t=1}^{|o_i|}\beta D_{kl}[\pi_\theta|\pi_{ref}]
\end{split}
$$